In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.express as px
import time

In [4]:
from sklearn.model_selection import train_test_split
import optuna.integration.lightgbm as lgb

# 環境設定

In [5]:
DATA_PATH = "./data"
RESULT_PATH = "./results"

# 関数定義

In [6]:
def cosine_similarity(y_true, y_pred):
    if isinstance(y_true, pd.Series):
        y_true = y_true.values.reshape(-1)
    if isinstance(y_pred, pd.Series):
        y_pred = y_pred.values.reshape(-1)
    score = np.dot(y_true, y_pred) / (np.linalg.norm(y_true) * np.linalg.norm(y_pred))
    return score

# 分析

## データ読み込み

In [8]:
train_data = pd.read_csv(Path(DATA_PATH, "train.csv"))
test_data = pd.read_csv(Path(DATA_PATH, "test.csv"))

## モデルの作成

In [9]:
# サンプリング（時間短縮のため）
sample_train_data = train_data.sample(frac=0.2)

In [10]:
# データ分割
# TODO: 時系列データであることを考慮すべき？
feature_cols = sample_train_data.drop(columns=["id", "target"]).columns
target_col = "target"
X_train, X_valid, y_train, y_valid = train_test_split(
    sample_train_data[feature_cols], 
    sample_train_data[target_col], 
    test_size=0.3
)
print(f'X_trainのshape:{X_train.shape}')
print(f'y_trainのshape:{y_train.shape}')
print(f'X_validのshape:{X_valid.shape}')
print(f'y_validのshape:{y_valid.shape}')

X_trainのshape:(92400, 695)
y_trainのshape:(92400,)
X_validのshape:(39600, 695)
y_validのshape:(39600,)


In [15]:
# データセットの作成
reg_train = lgb.Dataset(X_train, y_train)
reg_eval = lgb.Dataset(X_valid, y_valid, reference=reg_train)

In [19]:
# パラメータの準備
params = {
    'boosting_type':'gbdt',
    'objective':'regression',
    'metric': 'rmse',
    'random_state':0
}

In [20]:
# HPチューニング
start_time = time.time()
opt=lgb.train(
    params,
    reg_train,
    valid_sets = reg_eval,
    verbose_eval=False,
    num_boost_round = 5,
)
print(f"学習にかかった時間：{time.time() - start_time}")

[I 2023-08-18 01:05:34,424] A new study created in memory with name: no-name-2bc6065c-a72f-400a-9c2f-d1b5a0c2684e

feature_fraction, val_score: inf:   0%|                   | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019221 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4168
[LightGBM] [Info] Number of data points in the train set: 92400, number of used features: 695



feature_fraction, val_score: 0.996776:  14%|8     | 1/7 [00:05<00:34,  5.67s/it][I 2023-08-18 01:05:40,121] Trial 0 finished with value: 0.9967760977270483 and parameters: {'feature_fraction': 0.5}. Best is trial 0 with value: 0.9967760977270483.

feature_fraction, val_score: 0.996776:  14%|8     | 1/7 [00:05<00:34,  5.67s/it]

[LightGBM] [Info] Start training from score -0.002987
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.034317 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4168
[LightGBM] [Info] Number of data points in the train set: 92400, number of used features: 695
[LightGBM] [Info] Start training from score -0.002987



feature_fraction, val_score: 0.996776:  29%|#7    | 2/7 [00:11<00:27,  5.47s/it][I 2023-08-18 01:05:45,455] Trial 1 finished with value: 0.9968284563804962 and parameters: {'feature_fraction': 0.8}. Best is trial 0 with value: 0.9967760977270483.

feature_fraction, val_score: 0.996776:  29%|#7    | 2/7 [00:11<00:27,  5.47s/it]

...

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019474 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4168
[LightGBM] [Info] Number of data points in the train set: 92400, number of used features: 695
[LightGBM] [Info] Start training from score -0.002987



min_data_in_leaf, val_score: 0.994660:  80%|####8 | 4/5 [00:20<00:05,  5.16s/it][I 2023-08-18 01:11:31,303] Trial 66 finished with value: 0.9952372426453031 and parameters: {'min_child_samples': 25}. Best is trial 63 with value: 0.9946600119505157.

min_data_in_leaf, val_score: 0.994660:  80%|####8 | 4/5 [00:20<00:05,  5.16s/it]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019359 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4168
[LightGBM] [Info] Number of data points in the train set: 92400, number of used features: 695
[LightGBM] [Info] Start training from score -0.002987



min_data_in_leaf, val_score: 0.994660: 100%|######| 5/5 [00:25<00:00,  5.17s/it]

学習にかかった時間：362.05063104629517


In [21]:
opt.params

{'boosting_type': 'gbdt',
 'objective': 'regression',
 'metric': 'rmse',
 'random_state': 0,
 'feature_pre_filter': False,
 'lambda_l1': 0.003566907316652897,
 'lambda_l2': 1.9847715690369496,
 'num_leaves': 215,
 'feature_fraction': 0.52,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 100,
 'num_iterations': 5,
 'early_stopping_round': None}

In [26]:
# 精度
y_train_pred = opt.predict(X_train)
y_valid_pred = opt.predict(X_valid)
print(f"trainの精度：{cosine_similarity(y_train, y_train_pred)}")
print(f"validの精度：{cosine_similarity(y_valid, y_valid_pred)}")

trainの精度：0.3779359714929994
validの精度：0.09086984000249344


## テストデータの予測

In [27]:
# 予測
id_test = test_data["id"].values.reshape(-1)
X_test = test_data.drop(columns=["id"]).copy()
y_test_pred = opt.predict(X_test)

In [28]:
pd.DataFrame(y_test_pred)

,0
0,0.054365
1,0.015501
2,-0.007097
3,-0.007136
4,-0.083279
...,...
261656,0.028713
261657,0.036861
261658,-0.018638
261659,0.031034


In [29]:
# 保存
result = pd.DataFrame(data=y_test_pred, index=id_test, columns=["target"]).reset_index().rename(columns={"index": "id"})
result.to_csv(Path(RESULT_PATH, "20230817_result_with_optuna.csv"), index=False)